## Settings

In [3]:
from IPython.display import HTML

def warn_download_images():
  return HTML('Please set download_images to True in Getting Started to enable this feature.')
  
#@markdown Set *download_images* below to True if you want to train or evaluate models yourself, otherwise this notebook can work with downloaded results:
download_images = True #@param ["False", "True"] {type:"raw"}

#@markdown Set *download_more_images* below to True if you want to download more samples:
download_more_images = False #@param ["False", "True"] {type:"raw"}

I understood it late, but I finally saved the best-trained models directly on Google Drive. I do not need to think about it anymore and when I reach Google Colab's 12 consecutive hours limit ... I do not just lose my models, which happened to me a few times 😉

# Install Prerequisites

In [4]:
!pip install tensorflow-io
  
# Pretrained ConvNets for pytorch: NASNet, ResNeXt, ResNet, InceptionV4, InceptionResnetV2, Xception, DPN, etc.
!pip install -q --upgrade pretrainedmodels

# display live plots while training (loss, accuracy, ROC AUC...)
!pip install -q livelossplot==0.3.3

# display execution time for each cell
!pip install -q ipython-autotime
%load_ext autotime

# Install pydicom to extra image
!pip install -q pydicom

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.0/28.0 MB 47.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.5 MB/s eta 0:00:00
time: 6.15 s (started: 2023-04-22 14:27:05 +00:00)


In [5]:
import matplotlib.pyplot as plt
import tensorflow as tf

time: 6.37 s (started: 2023-04-22 14:27:11 +00:00)


# Download Data

In [6]:
def download(url, destination_folder='.'):
  !wget -nc -q --show-progress $url -P $destination_folder

data_root = "/content"

#download(f'https://isic-challenge-data.s3.amazonaws.com/2020/ISIC_2020_Training_JPEG.zip', data_root)
download(f'https://isic-challenge-data.s3.amazonaws.com/2020/ISIC_2020_Training_GroundTruth.csv', data_root)

ISIC_2020_Training_ 100%[===================>]   1.96M  --.-KB/s    in 0.1s    
time: 475 ms (started: 2023-04-22 14:27:18 +00:00)


In [6]:
import zipfile
import os
from tqdm import tqdm_notebook as tqdm

  # unzip train.zip, valid.zip, test.zip
with zipfile.ZipFile(os.path.join(data_root, f'ISIC_2020_Training_JPEG.zip'), 'r') as myzip:
  for file in tqdm(myzip.namelist(), desc=f'Extracting ISIC_2020_Training_JPEG.zip'):
    myzip.extract(member=file, path=data_root)

<ipython-input-6-fed44eea1554>:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for file in tqdm(myzip.namelist(), desc=f'Extracting ISIC_2020_Training_JPEG.zip'):


Extracting ISIC_2020_Training_JPEG.zip:   0%|          | 0/33127 [00:00<?, ?it/s]

time: 4min 43s (started: 2023-04-17 03:20:30 +00:00)


In [5]:
download(f'https://isic-challenge-data.s3.amazonaws.com/2020/ISIC_2020_Test_JPEG.zip', data_root)

ISIC_2020_Test_JPEG 100%[===================>]   6.71G  15.4MB/s    in 7m 32s  
time: 7min 33s (started: 2023-04-17 09:27:34 +00:00)


In [6]:
import zipfile
import os
from tqdm import tqdm_notebook as tqdm

with zipfile.ZipFile(os.path.join(data_root, f'ISIC_2020_Test_JPEG.zip'), 'r') as myzip:
  for file in tqdm(myzip.namelist(), desc=f'Extracting ISIC_2020_Test_JPEG.zip'):
    myzip.extract(member=file, path=data_root)

<ipython-input-6-ff0a772204da>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for file in tqdm(myzip.namelist(), desc=f'Extracting ISIC_2020_Test_JPEG.zip'):


Extracting ISIC_2020_Test_JPEG.zip:   0%|          | 0/10985 [00:00<?, ?it/s]

time: 1min 42s (started: 2023-04-17 09:35:08 +00:00)


# Explore Data

In [7]:
import tensorflow_io as tfio, os
import csv

# Read ground truth data into RAM
# Create images array, will store 2D array of [image_name, {data}]
images = []

with open(os.path.join(data_root, "ISIC_2020_Training_GroundTruth.csv")) as f:
  reader = csv.reader(f, delimiter=',')
  for row in reader:
    # Skip first row
    if row[0] == "image_name": continue

    # Add row to array
    images.append([
        row[0],
        {
         "id": row[1],
         "sex": row[2],
         "age": row[3],
         "anatom_site": row[4],
         "diagnosis": row[5],
         "benign_malignant": row[6],
         "target": row[7]
        }
    ])

def get_diagnosis(name):
  for image in images:
    if image[0] == name:
      return image[1]['diagnosis']
  
  return None

time: 193 ms (started: 2023-04-22 14:27:18 +00:00)


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
time: 20.1 s (started: 2023-04-22 14:27:19 +00:00)


In [9]:
diagnosis_types = {}
classes = []

for image in images:
  if (image[1]['diagnosis'] in diagnosis_types):
    diagnosis_types[image[1]['diagnosis']] += 1
  else:
    diagnosis_types[image[1]['diagnosis']] = 1
    classes.append(image[1]['diagnosis'])

for k, v in enumerate(diagnosis_types):
  print(f"{v}: {diagnosis_types[v]}")


unknown: 27124
nevus: 5193
melanoma: 584
seborrheic keratosis: 135
lentigo NOS: 44
lichenoid keratosis: 37
solar lentigo: 7
cafe-au-lait macule: 1
atypical melanocytic proliferation: 1
time: 174 ms (started: 2023-04-22 14:27:39 +00:00)


## Oversampling

We now have enough melanoma samples.  
But __still not enough seborrheic keratosis__. So I choose the __oversampling approach__ for this unbalanced class and I increase the number of observations which are just copies of existing samples.  

At each epoch, for a given phase, I duplicate image indexes randomly until I have as many samples in each class.

In [10]:
import random

type_to_oversample = "melanoma"

# Get total of all non-unknown images excluding type_to_oversample
total = 0

for k, v in enumerate(diagnosis_types):
  if v == "unknown": continue
  if v == type_to_oversample: continue

  total += diagnosis_types[v]

current_total = diagnosis_types[type_to_oversample]
print(f"Current {type_to_oversample} images to oversample: {current_total}/{total}")

type_image = []
for img in images:
  if img[1]['diagnosis'] != type_to_oversample: continue

  type_image.append(img)

while current_total != total:
  # Chose random image from sample to duplicate
  img = random.randint(0, current_total)
  img_content = images[img]

  # Duplicate image
  images.append(img_content)
  current_total += 1
   
print(f"Done, {type_to_oversample} now has {current_total} images to make a total sample size of {total + current_total}")

Current melanoma images to oversample: 584/5418
Done, melanoma now has 5418 images to make a total sample size of 10836
time: 148 ms (started: 2023-04-22 14:27:39 +00:00)


## Too small validation set
📝 In addition, the initial validation set is very small (150 samples) and will not be a good predictor of model performance. (another learning experience) 📝

So I __change the partition to have 80% in training (3412 samples) an 20% in validation (853)__.  

I of course keep the test set imbalanced for the challenge purpose.

In [11]:
# Update total image count
total = len(images)

# Create a new image set that is 10% of the current number of images
validation_set_size = round(total * 0.10)

# Create the new validation set
validation_set = []
for x in range(0, validation_set_size):
  # Chose random items from the main set to create the validation set
  img_id = random.randint(0, total - 1)
  img = images[img_id]

  validation_set.append(img)

print(f"Created new validation set with {len(validation_set)} images.")


Created new validation set with 3796 images.
time: 76.9 ms (started: 2023-04-22 14:27:39 +00:00)


# Data Augmentation

## Multi-crop

To multi-crop is to crop the input image into multi sub-images, input into network for classification, and average the result to increase the accuracy. PyTorch provides two multi-crop classes:
- [FiveCrop](https://pytorch.org/docs/stable/torchvision/transforms.html#torchvision.transforms.TenCrop) crops the given image into four corners and the central crop;
- and [TenCrop](https://pytorch.org/docs/stable/torchvision/transforms.html#torchvision.transforms.TenCrop) crops the given image into four corners and the central crop plus the flipped version of these ((horizontal flipping is used by default).  

You may have noticed in the code above the MultiCrop function. Here it is:

In [12]:
def MultiCrop(n_crops, image_size):

  if n_crops == 5:
    return transforms.FiveCrop(image_size)
  elif n_crops == 10:
    return transforms.TenCrop(image_size)
  elif n_crops == 20:
    return transforms.TwentyCrop(image_size)
  else:
    raise ValueError(f'Unsupported n_crops: {n_crops}')

time: 920 µs (started: 2023-04-22 14:27:39 +00:00)


As said earlier, [TenCrop](https://pytorch.org/docs/stable/torchvision/transforms.html#torchvision.transforms.TenCrop) flips image vertically __OR__ horizontally.  

So I created __TwentyCrop__ hereafter to flip image both vertically __AND__ horizontally. 💥

In [13]:
import numbers

from torchvision.transforms import functional as F

class TwentyCrop(object):
  """ TenCrop flips image vertically or horizontally. TwentyCrop does both.
  """

  def __init__(self, size):
      self.size = size
      if isinstance(size, numbers.Number):
          self.size = (int(size), int(size))
      else:
          assert len(size) == 2, "Please provide only two dimensions (h, w) for size."
          self.size = size

  def __call__(self, img):

    first_ten = F.ten_crop(img, self.size, vertical_flip=False)
    img = F.vflip(img)
    second_ten = F.ten_crop(img, self.size, vertical_flip=False)

    return first_ten + second_ten

  def __repr__(self):
      return self.__class__.__name__ + '(size={0})'.format(self.size)

time: 1.46 ms (started: 2023-04-22 14:27:39 +00:00)


📝Multi-crop takes as many times as there are crops, so of course it can be very slow! 😴  

So I enabled it only when metrics reach a predefined threshold as shown in code below. 📝

In [14]:
def get_n_crops(is_test_phase, metric, n_crops_stages={5: 0.875, 10: 0.890}):

  if is_test_phase:
    for crops, threshold in reversed(sorted(n_crops_stages.items())): 
      if metric >= threshold:
        return crops

  return 1

time: 883 µs (started: 2023-04-22 14:27:39 +00:00)


## Image Extraction and Pre-Processing

The images are provided in a medical-technology format known as DICOM, which encodes them with metadata.

The image needs to be extracted out into a file Tensorflow can work with. At the same time, the images are downscaled, to reduce the memory usage, and improve the performance of the algorithm.

In [15]:
import tensorflow_io as tfio

from PIL import Image

# Create new directories to store resized images
train_set_path = os.path.join(data_root, "train")
test_set_path = os.path.join(data_root, "test")
train_set_resized_path = os.path.join(data_root, "train_resized")
test_set_resized_path = os.path.join(data_root, "test_resized")
if not os.path.exists(train_set_resized_path): os.makedirs(train_set_resized_path)
if not os.path.exists(test_set_resized_path): os.makedirs(test_set_resized_path)

# Walk and resize all training images
print("Starting to resize all training imges...")
for root, dirs, files in os.walk(train_set_path):
  for name in files:
    # Open and resize image
    resized_image_path = os.path.join(train_set_resized_path, name)
    im = Image.open(os.path.join(train_set_path, name))
    # double size required by largest tested model (NASNetALarge)
    size = 331*2, 331*2
    if im.size != size:
      im = im.resize(size)
      print (name)
    im.save(resized_image_path)

print("Finished resize job.")



# def preprocess_images(folder, force=False):
  
#   if force or folder.endswith('_resized') and not os.path.exists(folder):
#     print('For performance purpose, images pre-processing (resizing) is required...')
    
#     for dir_name, path in data_dir.items():
#       for fn in tqdm(sorted(glob.glob(os.path.join(path, '*', '*.jpg'))), desc=('Resize ' + path.split(os.path.sep)[-1])):
#         resized_fn = fn.replace(dir_name, dir_name + '_resized')
#         if not os.path.exists(resized_fn):
#           if not os.path.exists(os.path.dirname(resized_fn)): 
#             os.makedirs(os.path.dirname(resized_fn))
#           try:
#             im = Image.open(fn)
#             # double size required by largest tested model (NASNetALarge)
#             size = 331*2, 331*2 
#             if im.size != size:
#               im = im.resize(size, Image.ANTIALIAS)
#             im.save(resized_fn)
#           except OSError as e:
#             print("error " + str(e))

Starting to resize all training imges...
Finished resize job.
time: 28.4 ms (started: 2023-04-22 14:27:39 +00:00)


In [16]:
# Copy resized training image to Google Drive
gdrive_dir = "/content/drive/MyDrive/trained_resized"

import shutil

# Walk all files
print("Starting to copy data to GDrive")
for root, dirs, files in os.walk(train_set_resized_path):
  for name in files:
    full_path = os.path.join(train_set_resized_path, name)
    shutil.copy(full_path, os.path.join(gdrive_dir, name))

print("Done copying data, check files are availabie in Google Drive before closing Colab.")

Starting to copy data to GDrive
Done copying data, check files are availabie in Google Drive before closing Colab.
time: 3.75 ms (started: 2023-04-22 14:27:44 +00:00)


In [17]:
import tensorflow_io as tfio

from PIL import Image

test_set_path = os.path.join(data_root, "test")
test_set_resized_path = os.path.join(data_root, "test_resized")
if not os.path.exists(test_set_resized_path): os.makedirs(test_set_resized_path)

print("Starting to resize all test imges...")
for root, dirs, files in os.walk(test_set_path):
  for name in files:
    # Open and resize image
    if name.endswith('.jpg'):
      resized_image_path = os.path.join(test_set_resized_path, name)
      im = Image.open(os.path.join(test_set_path, name))
      # double size required by largest tested model (NASNetALarge)
      size = 331*2, 331*2
      if im.size != size:
        im = im.resize(size)
        print (name)
        im.save(resized_image_path)

print("Finished resize job.")

Starting to resize all test imges...
Finished resize job.
time: 19.9 ms (started: 2023-04-22 14:27:46 +00:00)


In [24]:
# Copy resized training image to Google Drive
gdrive_dir = "/content/drive/MyDrive/test_resized"

import shutil

# Walk all files
print("Starting to copy data to GDrive")
for root, dirs, files in os.walk(test_set_resized_path):
  for name in files:
    full_path = os.path.join(test_set_resized_path, name)
    shutil.copy(full_path, os.path.join(gdrive_dir, name))

print("Done copying data, check files are availabie in Google Drive before closing Colab.")

Starting to copy data to GDrive
Done copying data, check files are availabie in Google Drive before closing Colab.
time: 1min 16s (started: 2023-04-17 11:38:48 +00:00)


In [39]:
# Copy GDrive images back into Colab space
gdrive_dir_train = "/content/drive/MyDrive/trained_resized"
gdrive_dir_test = "/content/drive/MyDrive/test_resized"

import shutil, os

print("Copying Training images back")
shutil.copytree(gdrive_dir_train, os.path.join(data_root, "trained_resized"))
print("Copying Test images back")
shutil.copytree(gdrive_dir_test, os.path.join(data_root, "test_resized"))

print("Redownload Ground truth into Colab")
download(f'https://isic-challenge-data.s3.amazonaws.com/2020/ISIC_2020_Training_GroundTruth.csv', data_root)

print("Done")

Copying Training images back


KeyboardInterrupt: ignored

time: 39min 21s (started: 2023-04-17 18:08:00 +00:00)


In [43]:
g_data_root = "/content/drive/MyDrive"
g_train_path = os.path.join(g_data_root, "trained_resized")
g_test_path = os.path.join(g_data_root, "test_resized")

time: 1.17 ms (started: 2023-04-22 14:29:26 +00:00)


# Create a Model

Let's first detect if we have a GPU available:

In [18]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

time: 3.61 ms (started: 2023-04-22 14:28:09 +00:00)


Having satisfactory performance with a model created from scratch would take days or weeks to train... and still you have to train multiples architectures and pick the best one.  

__[Transfer Learning](https://cs231n.github.io/transfer-learning/)__ is the reuse of a pre-trained model on a new problem. It is currently very popular in the field of Deep Learning because it enables you to train Deep Neural Networks with comparatively little data.
There are a some pre-trained Machine Learning models out there that became quite popular. One of them is the [DenseNet-161](https://arxiv.org/pdf/1608.06993.pdf) model, which was trained for the ImageNet “Large Visual Recognition Challenge”. In this challenge, participants had to classify images into 1000 classes, like “Zebra”, “Dalmatian”, and “Dishwasher”.

Despite the objects it was trained to classify are quite different compared to skin diseases images, the features detection part of such pretrained model are often reused to classify completely different images.  

I initially selected __[DenseNet-161](https://arxiv.org/pdf/1608.06993.pdf)__ as a good model to reuse because it has (had) __improved performance over others on ImageNet__ and it introduces an interesting architecture with __each layer taking all preceding feature-maps as input__. Moreover despite it has a lot of layers, the time to train a single epoch is very similar to simpler pretrained models.

Picture below shows the smaller DenseNet-121.

![title](https://cdn-images-1.medium.com/max/1000/1*BJM5Ht9D5HcP5CFpu8bn7g.png?raw=1)

We just need two modifications to the pretrained model (see configure_model function below):
 - 📝as skin diseases images are still different compared to cats, dogs, cars... I __freeze the parameters of the two first blocks (feature extraction), and I train the parameters of the two last blocks (fine-tuning)__ (Dense Block 3 and Dense Block 4). __This step is crucial!__ Otherwise the model caps at 80% accuracy in train set, and swings between 50-60% in test set. 📝
 - and then __replace the last fully connected layer (the classifier itself) so that it outputs 3 classes__ instead of 1000.

I tried two different final classifiers and retained the first one:
- one layer with a single Linear classifier;
- two layers, with batch normalization to prevent vanishing gradients and dropout for regularization.  

Both outputs raw scores that will later be transformed as probabilities using softmax.

In [19]:
import torchvision.models as models
import torch.nn as nn

def set_parameter_requires_grad(parameters, feature_extract):
  # set parameters that should be trained
  
  for param in parameters:
      param.requires_grad = not feature_extract

def configure_model(model, final_classifier, num_classes, feature_extract, skip_requires_grad = False):
  
  if not skip_requires_grad:
    set_parameter_requires_grad(model.parameters(), feature_extract)
    
  return nn.Linear(final_classifier.in_features, num_classes)

time: 1.01 ms (started: 2023-04-22 14:28:15 +00:00)


The initialize_model function below allows you to create and configure a few well-known models:

In [20]:
def get_image_size(model_name):
  
  model_name = model_name.lower()
  
  if model_name in ['inception3', 'inceptionv4', 'xception', 'inceptionresnetv2']:
    return 299
  elif model_name in ['nasnetalarge', 'pnasnet5large', 'polynet']:
    return 331
  else:
    return 224

time: 722 µs (started: 2023-04-22 14:28:17 +00:00)


In [21]:
import pretrainedmodels

def initialize_model(model_name, num_diagnoses, use_pretrained=False, feature_extract=True):
  
    model = None
    model_name = model_name.lower()

    if model_name == "resnet":
        """ Resnet152
        """
        model = models.resnet152(pretrained=use_pretrained)
        model.fc = configure_model(model, model.fc, num_diagnoses, feature_extract)

    elif model_name == "alexnet":
        """ Alexnet
        """
        model = models.alexnet(pretrained=use_pretrained)
        model.classifier[6] = configure_model(model, model.classifier[6], num_diagnoses, feature_extract)

    elif model_name == "vgg":
        """ VGG11_bn
        """
        model = models.vgg11_bn(pretrained=use_pretrained)
        model.classifier[6] = configure_model(model, model.classifier[6], num_diagnoses, feature_extract)

    elif model_name == "squeezenet":
        """ Squeezenet
        """
        model = models.squeezenet1_0(pretrained=use_pretrained)
        set_parameter_requires_grad(model.parameters(), feature_extract)
        model.classifier[1] = nn.Conv2d(512, num_diagnoses, kernel_size=(1,1), stride=(1,1))
        model.num_classes = num_diagnoses
    elif model_name == "densenet":
        """ Densenet
        """
        model = models.densenet161(pretrained=use_pretrained)
        model.classifier = configure_model(model, model.classifier, num_diagnoses, feature_extract)

    elif model_name == "inception3":
        """ Inception v3
        """
        model = models.inception_v3(pretrained=use_pretrained)
        # Handle the auxilary net
        model.AuxLogits.fc = configure_model(model, model.AuxLogits.fc, num_diagnoses, feature_extract)
        # Handle the primary net
        model.fc = configure_model(model, model.fc, num_diagnoses, feature_extract, skip_requires_grad=True)
        
    elif model_name in pretrainedmodels.__dict__['model_names']:
        """ pretrainedmodels
        """        
        model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained=('imagenet' if use_pretrained else None))
        model.last_linear = configure_model(model, model.last_linear, num_diagnoses, feature_extract)

    else:
        print(f"Invalid model name: {model_name}, exiting...")
        exit()

    # Send the model to GPU if any
    return model.to(device), get_image_size(model_name)

time: 1.37 s (started: 2023-04-22 14:28:19 +00:00)


## Feature extraction or fine-tuning

Choosing the parameters for feature extraction or fine-tuning (first_trained_module in code below) is __almost random guess__...  

📝So I introduced(*) __fine_tuning_module_rounds__: while training, it automatically adds one more module for fine-tuning if ROC AUC does not increase within given n epochs rounds. It is set to Infinite by default for the behavior to be consciously chosen.📝  

<i>(*) Most of my models where trained with a bit of "guess". Only the last two trials had fine_tuning_module_rounds enabled.</i>

In [22]:
import numpy as np

def find_first_trained_module(model, start_module_index=np.Inf, start_module_name=None):
  
  for i, (name, module) in enumerate(reversed(list(model.named_modules()))):
    set_parameter_requires_grad(module.parameters(), feature_extract=False)
    if (start_module_name == name or i > start_module_index) and len(list(module.parameters())) > 0:       
      return i, name, get_optimizer(model, name if start_module_name is None else start_module_name)

def set_trained_modules(model, optimizer, epoch, best_epoch, since_epoch, first_trained_module, first_trained_module_i, best_auc, test_phase, fine_tuning_module_rounds=np.Inf):
  
  if (epoch == fine_tuning_module_rounds + best_epoch + 1) or (best_epoch < since_epoch and epoch == fine_tuning_module_rounds + since_epoch + 1) :
    since_epoch = epoch
    first_trained_module_i, first_trained_module, optimizer = find_first_trained_module(model, first_trained_module_i + 1)
    
  logging.info(get_params_to_train_str(model, first_trained_module_i, f' (since epoch {since_epoch:2.0f})', 
                                       fine_tuning_rounds_str = '' if (fine_tuning_module_rounds==np.Inf or epoch==1) else f' ↑ Fine-tuning goes up one module if {test_phase} ROC AUC < {best_auc:.3f} at end of epoch {(best_epoch+fine_tuning_module_rounds) if best_epoch<since_epoch else (since_epoch+fine_tuning_module_rounds):2.0f} ↑ '))

  return since_epoch, first_trained_module_i, first_trained_module, optimizer

time: 4.55 ms (started: 2023-04-22 14:28:22 +00:00)


In [23]:
def trail_str(str, max_length, last_chars=''):
  
  if len(str) > max_length-2-len(last_chars):
    return str[0:max_length-3] + '...' + last_chars
  return str.ljust(max_length-len(last_chars)) + last_chars
    
def get_params_to_train_str(model, first_trained_module_i, since_epoch_str, fine_tuning_rounds_str=None):
  
  modules = [(('= ' if i > first_trained_module_i else '~ ') + f'{name}') for (i, (name, module)) in enumerate(reversed(list(model.named_modules()))) if name]                                                                                                             
  
  params_to_train, n_params = get_params_to_train(model)  
  total_parameters = len(list(model.parameters()))
  
  feature_extract_modules = modules[first_trained_module_i+1:first_trained_module_i+3] + ['= ...'] + modules[::-1][0:3][::-1]
  feature_extract_modules[0] = trail_str(feature_extract_modules[0], 40)                + f' ↑ Fixed feature extractor on {str(total_parameters-n_params).rjust(4)} parameters out of {str(total_parameters).rjust(4)} in modules above   ↑'
  fine_tuning_modules = modules[0:3] + ['~ ...'] + modules[first_trained_module_i-1:first_trained_module_i+1]
  fine_tuning_modules[-1] = trail_str(fine_tuning_modules[-1], 40, since_epoch_str) + f' ↓ Fine-tuning                {str(n_params).rjust(4)} parameters out of {str(total_parameters).rjust(4)} in modules below   ↓'
  
  return '\n'.join(reversed(feature_extract_modules)) \
      + f'\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~{fine_tuning_rounds_str}~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\n' \
       + '\n'.join(reversed(fine_tuning_modules))

def get_params_to_train(model, max_params=6):
  
  params_to_train = list(get_params_to_optimize(model).keys())
  n_params = len(params_to_train)
  
  if n_params > max_params:
    params_to_train = params_to_train[0:max_params//2] + ['...'] + params_to_train[::-1][0:max_params//2][::-1]
  
  return params_to_train, n_params

time: 2.78 ms (started: 2023-04-22 14:28:24 +00:00)


## Loss function

__CrossEntropyLoss__ is the appropriate loss function for a classification model that outputs raw scores for each class.  

__Melanoma is the deadliest form of cancer__. Despite my samples will correctly be balanced at each iteration, I want to __penalize by 50% the loss on melanoma__.

In [24]:
#rescaling weight given to each class
weight = [1.5, 1, 1]
criterion = nn.CrossEntropyLoss(torch.Tensor(weight).to(device))

time: 1.08 ms (started: 2023-04-22 14:28:27 +00:00)


## Optimizer

In [25]:
def get_params_to_optimize(model):
  # Gather the parameters to be optimized/updated in this run. 
  params_to_update = {}
  for name,param in model.named_parameters():
      if param.requires_grad == True:
          params_to_update[name] = param
  
  return params_to_update

time: 685 µs (started: 2023-04-22 14:28:28 +00:00)


Whatever the optimizer, he needs to know which parameters of the model he should optimize. If we are finetuning we will be updating all parameters. However, if we are doing feature extract method, we will only update the parameters that we have just initialized, i.e. the parameters with requires_grad is True.

I selected __Adam optimizer__ because it achieves good results fast. This algorithm is an extension to stochastic gradient descent that has recently seen broader adoption for deep learning applications in computer vision. It combines the advantages of two other extensions of stochastic gradient descent : AdaGrad and RMSProp.

In [26]:
import torch.optim as optim

def get_optimizer(model, first_fine_tuned_module):

  return optim.Adam(get_optimizer_params_group(model, first_fine_tuned_module), lr=0.001, weight_decay=1e-6)

time: 669 µs (started: 2023-04-22 14:28:30 +00:00)


📝 It’s common to use a smaller learning rate for ConvNet weights that are being fine-tuned, in comparison to the (randomly-initialized) weights for the new linear classifier that computes the class scores of your new dataset. This is because we expect that the ConvNet weights are relatively good, so we don’t wish to distort them too quickly and too much (especially while the new Linear Classifier above them is being trained from random initialization).📝  

Although the usual practice is to reduce the initial learning rate by 10 times compared to that used for scratch training, I preferred to use different learning rates according to the universality / specificity of what is captured by each layer. Indeed, the first layers capture universal features while the latter capture dataset-specific ones.

In [27]:
def get_optimizer_params_group(model, first_fine_tuned_module, classifier_lr=0.001, highest_layer_lr=2*1e-4, lowest_layer_lr=0.5e-4, weight_decay=1e-6):
  
  modules = list(get_top_modules(model, first_fine_tuned_module))
  modules = [m for m in modules if not any((m['name']+'.') in m2['name'] for m2 in modules)]
  learning_rates = list(np.linspace(lowest_layer_lr, highest_layer_lr, len(modules)-1)) + [classifier_lr]
  
  for m, lr in zip(modules, learning_rates):
    yield {'group_name': m['name'], 'params': m['module'].parameters(), 'lr': lr, 'weight_decay': weight_decay}

time: 1.01 ms (started: 2023-04-22 14:28:31 +00:00)


In [28]:
from collections import OrderedDict
  
def get_top_modules(model, first_fine_tuned_module):
            
  module_names = [f'{name}' for (i8, (name, module)) in enumerate(list(model.named_modules())) if (name and len(list(module.parameters()))>0)]
  top_modules = []
  
  depth=0
  while len(top_modules)<10 and depth<4:
    depth+=1
    top_modules = list(OrderedDict.fromkeys([get_top_module_name(name, depth) for name in module_names]))

  first_fine_tuned_top_module = sorted([m for m in top_modules if m in first_fine_tuned_module], key=len)[-1]

  fine_tuned_modules = top_modules[top_modules.index(first_fine_tuned_top_module):]
  for name in fine_tuned_modules:
    yield {'name': name, 'module': get_sub_module(model, name)}

def get_top_module_name(name, depth=1):
  
  if depth==0: 
    return name
  else:
    return '.'.join(name.split('.')[0:depth]) if name.count('.')>=(depth-1) else get_top_module_name(name, depth-1)
  
def get_sub_module(module, name):
  
  name = name.split('.')
  module = module[int(name[0])] if name[0].isdigit() else getattr(module, name[0])
  
  return module if len(name)==1 else get_sub_module(module, '.'.join(name[1:]))

time: 3.1 ms (started: 2023-04-22 14:28:32 +00:00)


## Choosing the right metric 
What is the single metric to take into account to determine if the model is good?

For classification problems, the following metrics are often usefull:
- Precision-Recall
- ROC-AUC
- Accuracy
- Log-loss

In this challenge, the decision is easy because the performance is measured against the __mean ROC-AUC of melanoma and seborrheic keratosis__.

In [29]:
from sklearn.metrics import roc_curve, auc

def get_roc_auc(y_true, y_pred, plot=False):
    """
    return ROC AUC Score for melanoma, seborrheic_keratosis, and their mean
    """

    # initialize dictionaries and array
    fpr = dict()
    tpr = dict()
    roc_auc = np.zeros(3)
    
    # prepare for figure
    if plot:
      plt.figure()
      colors = ['aqua', 'cornflowerblue']

    # for both classification tasks (categories 1 and 2)
    for i in range(2):
        # obtain ROC curve
        fpr[i], tpr[i], _ = roc_curve(y_true[:,i], y_pred[:,i])
        # obtain ROC AUC
        roc_auc[i] = auc(fpr[i], tpr[i])
        if plot:
          # plot ROC curve
          plt.plot(fpr[i], tpr[i], color=colors[i], lw=2,
                   label='ROC curve for task {d} - class {c} (area = {f:.3f})'.format(d=i+1, c=('melanoma' if i==0 else 'other'), f=roc_auc[i]))
          
    # get score for category 3
    roc_auc[2] = np.average(roc_auc[:2])
    
    if plot:
      # format figure
      plt.plot([0, 1], [0, 1], 'k--', lw=2)
      plt.xlim([0.0, 1.0])
      plt.ylim([0.0, 1.05])
      plt.xlabel('False Positive Rate')
      plt.ylabel('True Positive Rate')
      plt.title(f'ROC curves - (mean area = {roc_auc[2]:.3f})')
      plt.legend(loc="lower right")
      ax = plt.gca()
      ax.set_facecolor('white')
      plt.show()

      # print scores
      for i in range(3):
          print('Category {d} Score: {f:.3f}'. format(d=i+1, f=roc_auc[i]))
        
    return roc_auc

time: 1.28 s (started: 2023-04-22 14:28:35 +00:00)


Have we achieved a higher ROC AUC metric in the validation set? Let's save it!

In [30]:
import copy

def save_model(save_path, model, best_epoch_auc):
  # we achieved a higher ROC AUC metric in validation set!
  # let's save it
  
  if save_path is not None:
    logging.info('=> Saving model')
    torch.save(model.state_dict(), save_path.replace('.pt', f'_{best_epoch_auc:.4f}.pt'))
  
  best_model_wts = copy.deepcopy(model.state_dict())                    
  
  return best_model_wts, best_epoch_auc

time: 659 µs (started: 2023-04-22 14:28:38 +00:00)


... and take the opportunity to write the code to reload this saved model.

In [31]:
def load_model(state_dict_fn):
  
  if os.path.exists(os.path.join(os.getcwd(), state_dict_fn)):
    # already downloaded
    state_dict_fn = os.path.join(os.getcwd(), state_dict_fn)
  elif os.path.exists(os.path.join(google_drive_shared_path, state_dict_fn)):
    # available on my own google drive
    state_dict_fn = os.path.join(google_drive_shared_path, state_dict_fn)
  else:
    state_dict_fn = os.path.join(os.getcwd(), state_dict_fn)
    download_file(state_dict_fn, quiet=True)
  
  model, image_size = initialize_model(get_model_name(state_dict_fn).lower(), len(classes))
  model.load_state_dict(torch.load(state_dict_fn))
  
  return model

time: 988 µs (started: 2023-04-22 14:28:39 +00:00)


In [32]:
def get_model_name(fn):
  
  model_name = fn.split(os.path.sep)[-1].split('_')[0].split('.')[0]
  
  return f'{model_name}154' if model_name.lower()=='senet' else model_name

time: 768 µs (started: 2023-04-22 14:28:43 +00:00)


... and finally release the model when we don't need it anymore, so that it frees GPU memory.

In [33]:
def release_model(model):
  # free GPU memory
  
  if model is not None:
    del model
    torch.cuda.empty_cache()

time: 546 µs (started: 2023-04-22 14:28:44 +00:00)


# Train the Model

## Helper functions

### Logging
I want to log some information before, after and while training. Here is what I needed:

In [34]:
import glob
def get_model_path(model_name, extension):
  # return a unique name for logging and saving model
  
  i = 1
  path = g_train_path if os.path.exists(g_train_path) else os.getcwd()
  while len(glob.glob(os.path.join(path, f'{model_name}_{i}*.{extension}'))):
    i+=1
  return os.path.join(path, f'{model_name}_{i}.{extension}')

time: 869 µs (started: 2023-04-22 14:28:45 +00:00)


In [35]:
import logging

def init_log(model_name):
  
  # reset handlers
  rootLogger = logging.getLogger()
  rootLogger.handlers = []
  rootLogger.setLevel(logging.INFO)

  # log to file
  save_path = get_model_path(model_name, 'pt')
  fileHandler = logging.FileHandler(save_path.replace('.pt', '.log'))
  fileHandler.terminator = '\r\n'
  rootLogger.addHandler(fileHandler)

  # log to console
  consoleHandler = logging.StreamHandler()
  rootLogger.addHandler(consoleHandler)
  
  return save_path

time: 996 µs (started: 2023-04-22 14:28:46 +00:00)


In [36]:
from collections import Counter
  
def log_model_summary(model, criterion, optimizer, phase_data_dirs_resized, over_sampling, cv, start=True, time_elapsed=0):
  
  model_name = model.__class__.__name__
  first_word = 'Start' if start else 'End'
  logging.info(f'{first_word} training {model_name} model')
  
  if not start:
    logging.info('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
  
  y = get_true_labels(phase_data_dirs_resized['train'])
  counter = Counter(y)
  # counter = len(y)
  logging.info(f'  Class distribution ({np.sum(counter.values())}): ' + ', '.join(f'{count} ' + classes[int(disease)] for disease, count in counter.items()))
  logging.info(f'  Oversampling: {over_sampling}')
  logging.info(f'  Cross validation: {cv}')
  
  logging.info(f'  Loss function: {criterion} - Weights: {criterion.weight}')
  logging.info(f'  Optimizer: {optimizer}')
  
  # train_transforms = get_transform(model_name, 'train')
  # logging.info(f'\n  Train tranforms: {train_transforms}')
  logging.info('')

time: 1.62 ms (started: 2023-04-22 14:28:49 +00:00)


### Live plotting
_"Don't train deep learning models blindfolded! Be impatient and look at each epoch of your training!"_  

I forked [LiveLossPlot](https://github.com/sebastienlange/livelossplot) to enhance it and [add support for custom series](https://github.com/stared/livelossplot/pull/46) and [add support for marker on higher/lower scores](https://github.com/stared/livelossplot/pull/48).

While ROC AUC is the choosen metric, I also plot after each epoch log-loss and accuracy.

In [37]:
def update_liveplot(logs, phase, epoch_loss, epoch_acc, epoch_auc):
  ## update data for LiveLossPlot
    
  logs[f'{phase}_log loss'] = epoch_loss
  logs[f'{phase}_accuracy'] = epoch_acc
  logs[f'{phase}_ROC AUC'] = epoch_auc[2]
  
  return logs

time: 729 µs (started: 2023-04-22 14:28:52 +00:00)


I also display on the progress bar log-loss, accuracy and even per-class accuracy after each batch... That's very addictive 😃

In [38]:
def get_running_status(phase, running_loss, running_accuracy):
  # update progress bar with current loss and accuracy
  # and even pre-class accuracy
  
  progresses = []
  progresses.append(f'{phase}_loss={running_loss/sum(np.array(list(running_accuracy.values()))[:,1]):.3f}')
  progresses.append(f'{phase}_acc={compute_accuracy(running_accuracy):.3f}')
  
  for disease in classes:
    progresses.append(f'{disease[0:5]}_acc={running_accuracy[disease][0]/running_accuracy[disease][1]:.3f}')

  return ', '.join(progresses)

time: 997 µs (started: 2023-04-22 14:28:54 +00:00)


### Computing stats
The final classifier output raw scores.  

To compute ROC AUC score, I need probabilities and one-hot encoded true labels:

In [39]:
def compute_batch_metrics(inputs, labels, outputs, loss, running_loss, one_hot_labels, outputs_probabilities, running_accuracy):
  # for a given batch it computes the one_hot_labels (one hot encoding)
  # and the probabilities for each class
  
  # use LabelBinarizer instead ? https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelBinarizer.html#sklearn.preprocessing.LabelBinarizer
  one_hot_labels = np.append(one_hot_labels, one_hot_encoding(labels.data.cpu().numpy()), axis=0)

  # track running prediction probabilities
  batch_probabilities = torch.nn.Softmax(dim=1)(outputs.data).cpu().numpy()
  outputs_probabilities = np.append(outputs_probabilities, batch_probabilities, axis=0)
  
  # track running loss
  running_loss += loss.item() * inputs.size(0)
  
  # track running accuracy
  _, predicted_labels = torch.max(outputs, 1)  
  for i, disease in enumerate(classes):
    # nb corrects for disease
    running_accuracy[disease][0] += torch.sum(predicted_labels[labels.data==i]==labels.data[labels.data==i]).cpu().numpy()
    # nb processed for disease
    running_accuracy[disease][1] += labels.data[labels.data==i].size(0)
  
  return running_loss, one_hot_labels, outputs_probabilities, running_accuracy

time: 3.24 ms (started: 2023-04-22 14:28:56 +00:00)


In [40]:
def one_hot_encoding(data):
  
  one_hot = np.zeros((data.size, 3))
  one_hot[np.arange(data.size),data] = 1
  
  return one_hot

time: 678 µs (started: 2023-04-22 14:28:58 +00:00)


In [41]:
import random

def balance_indices(epoch_indices, y, subset=1):
  # it uses oversampling to increase the size of imbalanced classes by randomly selecting more samples until all classes have same images count
  # subset allows a "cold" start: reusing a portion of images of most represented class
  
  images_per_class = {i:[] for i in range(len(classes))}
          
  for i in epoch_indices:
    images_per_class[y[i]].append(i)
  
  counts = [len(indexes) for indexes in images_per_class.values()] 
  max_samples_per_class = int(np.max(counts) * subset)
  if subset < 1:
    logging.info(f'Cold start: reusing {subset*100:.1f}% of images of most represented class')
  
  for i in range(len(classes)):
    if len(images_per_class[i]) > max_samples_per_class:
      images_per_class[i] = random.sample(images_per_class[i], max_samples_per_class)
    else:
      for j in range(max_samples_per_class-len(images_per_class[i])):
        over_sampled_i = np.random.randint(0, counts[i])
        images_per_class[i].append(images_per_class[i][over_sampled_i])
      
  return sorted([item for sublist in images_per_class.values() for item in sublist])

time: 1.47 ms (started: 2023-04-22 14:28:59 +00:00)


## Cross-validation

Finally I use the __[RepeatedStratifiedKFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RepeatedStratifiedKFold.html) cross validator__ with 5 splits for the following reasons:
- I do not want to leave aside my precious 853 validation samples... they will also be used in training;
- the folds are made by preserving the percentage of samples for each class;
- every 5 epochs, the folds are resampled.

In [87]:
#phase_data_dirs_resized = {'train': images, 'validation': validation_set}

phase_data_dirs_resized = {'train': [], 'validation': []}

for image in images:
  phase_data_dirs_resized['train'].append(
      os.path.join(g_train_path, image[0]) + ".jpg"
  )

# Repeat above for validation set
for image in validation_set:
  phase_data_dirs_resized['validation'].append(
    os.path.join(g_train_path, image[0]) + ".jpg", # Path to image
  )

def get_true_labels(data_dirs):
  # it returns class_id for each sample in the provided folders [0 0 0 1 1 2]
  
  y = []
  for img_path in data_dirs:
    filename = os.path.basename(img_path).split(".")[0] # Get image ID from path

    diagnosis = get_diagnosis(filename)
    # y = np.append(y, diagnosis)

    # y.append(tuple([
    #     filename,
    #     diagnosis
    # ]))

    # Get index of class from classes list as class_id
    class_id = int(classes.index(diagnosis))

    y = np.append(y, np.full(int(len((filename, diagnosis))), class_id))

  return y

time: 68.3 ms (started: 2023-04-22 15:41:03 +00:00)


In [45]:
from sklearn.model_selection import RepeatedStratifiedKFold, StratifiedKFold

def get_kfold_splitter(phase_y, num_epochs, cv=5, test_split=10):
  # return a lambda to create new k-folds every k iterations
  # the lambda will they return the training_indices and validation_indices
    
  X = list(range(len(phase_y['train'])))
  y = phase_y['train']
  print(f"Length of phase_y train: {len(phase_y['train'])}")
  if test_split is not None:
    # return one single split to have train_and_validation indices, and test split (will never be trained)
    train_valid_indices, test_indices = next(iter(StratifiedKFold(n_splits=test_split).split(X, y)))
    y = [phase_y['train'][i] for i in train_valid_indices]
  else:
    train_valid_indices = X
    test_indices = list(range(len(phase_y['test']))) if 'test' in phase_y else []
  
  rskf = RepeatedStratifiedKFold(n_splits=cv, n_repeats=int(num_epochs/5))
  
  return lambda: [get_phases_indices(train_indices, valid_indices, test_indices) for (train_indices, valid_indices) in rskf.split(train_valid_indices, y)]

time: 23.1 ms (started: 2023-04-22 14:29:58 +00:00)


In [46]:
def get_split_strategy(phase_y, num_epochs, cv=5, test_split=10):
  
  if cv is None:
    return lambda: [{phase: list(range(len(y))) for (phase, y) in phase_y.items()} for idx in range(num_epochs)]
  
  return get_kfold_splitter(phase_y, num_epochs, cv, test_split) 

time: 933 µs (started: 2023-04-22 14:30:00 +00:00)


In [47]:
def get_phases_indices(train_indices, valid_indices, test_indices):
  
  phases_indices = {'train': train_indices, 'valid': valid_indices}
  
  if len(test_indices) > 0:
    phases_indices['test'] = test_indices
  
  return phases_indices

time: 727 µs (started: 2023-04-22 14:30:01 +00:00)


In [101]:
from torch.utils.data import SubsetRandomSampler

def get_data_loader(phase, model, folders, indices=None, phase_y=None, over_sampling=False, best_auc=1, n_crops=1, resize=0.75):

  dataset = phase_data_dirs_resized
  #get_dataset(get_transform(model.__class__.__name__, phase, resize, n_crops=n_crops), folders)
  print (dataset)
  data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size[phase], sampler=sampler, num_workers=num_workers)
  print (data_loader)
  
  sampler = None
  
  if indices is not None:
    sampler = SubsetRandomSampler(balance_indices(indices, phase_y['train'], 1) if (over_sampling and phase=='train') else indices)        
  
  return torch.utils.data.DataLoader(dataset, batch_size=batch_size[phase], sampler=sampler, num_workers=num_workers)

time: 1.36 ms (started: 2023-04-22 16:57:40 +00:00)


In [92]:
def get_dataset(transform, dirs):
  # it returns as many ImageFolder as we have paths in a given phase 
  # e.g. If cross-validation is used and I have downloaded more data, my training set could have images from 'train', 'valid', and 'more' folders

   dataset = "/content/drive/MyDrive/trained_resized/train_resized"
  
   return dataset

time: 673 µs (started: 2023-04-22 15:56:48 +00:00)


In [50]:
import torchvision.transforms as transforms

def get_transform(model_name, phase, resize=0.75, n_crops=5):

  image_size = get_image_size(model_name)

  if phase=='train':
    return transforms.Compose([
                                transforms.RandomResizedCrop(image_size, scale=(0.7,1)),
                                transforms.RandomApply([DrawHair()], p=0.25),
                                transforms.RandomApply([DrawShape()], p=0.25),
                                transforms.RandomAffine(degrees=30, translate=(0.2,0.2), scale=(0.8,1.2), shear=(-8,8)),
                                transforms.ColorJitter(brightness=0.25, contrast=0.25, saturation=0.25, hue=0.1),
                                ImgAugTransform(iaa.Sometimes(0.2, iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.15*255), per_channel=0.5, name='GaussianNoise'))),
                                ImgAugTransform(iaa.Sometimes(0.2, iaa.GaussianBlur(sigma=(0, 1.0), name='GaussianBlur'))),
                                ImgAugTransform(iaa.Sometimes(0.2, iaa.CoarseDropout((0.03, 0.15), size_percent=(0.05, 0.2), per_channel=0.2))),
                                ImgAugTransform(iaa.Sometimes(0.2, iaa.JpegCompression(0.87))),
                                #ImgAugTransform(iaa.Sometimes(0.2, iaa.Sharpen(alpha=(0, 1.0), lightness=(0.75, 1.5), name='Sharpen'))),
                                transforms.RandomHorizontalFlip(),
                                transforms.RandomVerticalFlip(),
                                transforms.ToTensor(),
                                get_normalize(model_name)])
  elif n_crops==1:    
    return transforms.Compose([
                                transforms.Resize(int(image_size/resize)),
                                transforms.CenterCrop(image_size),
                                transforms.ToTensor(),
                                get_normalize(model_name)])
  else:
    return transforms.Compose([
                                transforms.Resize(int(image_size/resize)),
                                MultiCrop(n_crops, image_size),
                                transforms.Lambda(lambda crops: torch.stack([transforms.ToTensor()(crop) for crop in crops])),
                                transforms.Lambda(lambda crops: torch.stack([get_normalize(model_name)(crop) for crop in crops]))
    ])
  
# All pre-trained models expect input images normalized in the same way,
# i.e. mini-batches of 3-channel RGB images of shape (3 x H x W),
# where H and W are expected to be at least 224 (299 for inception)

def get_normalize(model_name):
  
  if model_name.lower() in ['inceptionv4', 'pnasnet5large', 'inceptionresnetv2', 'xception']: #'NasNetALarge']
    return transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  
    # My NasNetALarge models were erroneoously trained with values below (rather than above) so I did not fix my bug otherwise I had to train it again
    
  return transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

time: 1.59 ms (started: 2023-04-22 14:30:06 +00:00)


## Train functions

In [51]:
from PIL import Image, ImageDraw

# draw random lines for fake hairs
class DrawHair:

  def __call__(self, img):

    draw = ImageDraw.Draw(img)

    for i in range(np.random.randint(5,20)):
        draw.line((np.random.randint(0,img.size[0]),
                   np.random.randint(0,img.size[1]),
                   np.random.randint(0,img.size[0]),
                   np.random.randint(0,img.size[1])), fill=0, width=np.random.randint(1,7))
    del draw

    return img

time: 966 µs (started: 2023-04-22 14:30:08 +00:00)


In [52]:
# draw random colored rectangles or ellipse for fake "patches"
class DrawShape:

  def __call__(self, img):

    draw = ImageDraw.Draw(img)

    corner = np.random.randint(1,5)
    coord = ((np.random.randint(0,img.size[0]/8) + img.size[0]/2* (corner==2 or corner==4), np.random.randint(0,img.size[1]/8 + img.size[0]/2* (corner==3 or corner==4))), 
                    (np.random.randint(0,img.size[0]/2) + img.size[0]/2* (corner==2 or corner==4), np.random.randint(0,img.size[1]/2 + img.size[0]/2* (corner==3 or corner==4))))
    if np.random.rand()<0.5:
        draw.ellipse(coord, fill=np.random.randint(0,240))
    else:
        draw.rectangle(coord, fill=np.random.randint(0,240))

    del draw

    return img

time: 1.41 ms (started: 2023-04-22 14:30:11 +00:00)


In [53]:
import imgaug as ia

from imgaug import augmenters as iaa

# wrapper around imgaug compatible with pytorch transforms
class ImgAugTransform:
  def __init__(self, aug):
    self.aug = aug
      
  def __call__(self, img):
    img = np.array(img)
    return Image.fromarray(np.uint8(self.aug.augment_image(img)))
  
  def __repr__(self):
    return self.__class__.__name__ + ' ' + str(self.aug)

time: 1.4 s (started: 2023-04-22 14:30:12 +00:00)


In [54]:
# Get images from phases and determine GDrive Path

image_phases = {
    "train": [],
    "validate": []
}

for image in images:
  image_phases['train'].append(os.path.join(g_train_path, image[0]) + ".jpg")

for image in validation_set:
  image_phases['validate'].append(os.path.join(g_train_path, image[0]) + ".jpg")

time: 70.6 ms (started: 2023-04-22 14:30:14 +00:00)


In [55]:
import torch

from torchvision import datasets
from PIL import ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES = True

# number of subprocesses to use for data loading
num_workers = 0

# how many samples per batch to load
batch_size = {phase: 32 if phase=='train' else 16 for phase in ['test', 'train', 'validate']}

image_folders = {phase: datasets.ImageFolder(g_train_path, transform=get_transform('densenet', phase)) for phase in ['train', 'validate']}
loaders = {phase: torch.utils.data.DataLoader(g_train_path, batch_size=batch_size[phase], num_workers=num_workers, shuffle=(phase=='train')) for phase in ['train', 'validate']}

time: 52.4 s (started: 2023-04-22 14:30:15 +00:00)


__Here we are finally!__  😅  

The function below will train the model for a given number of epochs.

After each epoch, it evaluates the model against the validation set, and its parameters are saved if validation ROC AUC has increased. It's a good way to do early stopping when the model starts to overfit to the training set. 

While training, it plots live training and validation loss, accuracy and ROC AUC.

In [89]:
import time

from livelossplot import PlotLosses

def train_model(model, criterion, phase_data_dirs_resized, over_sampling=True, num_epochs=25, cv=5, test_split=None, first_trained_module=None, best_auc = 0.0, fine_tuning_module_rounds=np.Inf, n_crops_stages={5: 0.875, 10: 0.890}):
  print("Start model init")
  model, optimizer, first_trained_module_i, save_path, starting_time, best_model_wts, phase_y, split_strategy, live_loss_plot \
    = init_training(model, criterion, phase_data_dirs_resized, num_epochs, over_sampling, cv, test_split, first_trained_module) 
  print("Done")
  best_epoch = 1
  since_epoch = 1

  try:        
    for epoch, phases_indices in enumerate(split_strategy(), 1):

      logs = {}
      
      current_best_auc = best_auc
      since_epoch, first_trained_module_i, first_trained_module, optimizer = set_trained_modules(model, optimizer, epoch, best_epoch, since_epoch, first_trained_module, first_trained_module_i, best_auc, list(phases_indices.keys())[-1], fine_tuning_module_rounds)

      # Each epoch has a training and validation phase
      for phase, indices in phases_indices.items():
       
        data_loader = init_epoch(model, phase, phase_data_dirs_resized["train"], indices, phase_y, over_sampling, epoch, num_epochs, cv, phase==list(phases_indices.keys())[-1], n_crops_stages=n_crops_stages, best_auc=best_auc)

        # run a single epoch
        logs, best_auc, best_model_wts, _, _ = run_epoch(phase, data_loader, model, criterion, optimizer, logs, best_auc, best_model_wts, save_path.replace('.pt', f'_{first_trained_module}.pt'), save_phase = list(phases_indices.keys())[-1])
        
      if best_auc > current_best_auc:
        best_epoch = epoch
        current_best_auc = best_auc         

      terminate_epoch(live_loss_plot, logs)

  except KeyboardInterrupt:
      logging.info('Training interrupted')
      pass

  log_model_summary(model, criterion, optimizer, phase_data_dirs_resized, over_sampling, cv, False, time.time() - starting_time)

  # load best model weights
  model.load_state_dict(best_model_wts)

  return model

time: 2.64 ms (started: 2023-04-22 15:44:58 +00:00)


In [57]:
def init_training(model, criterion, phase_data_dirs_resized, num_epochs, over_sampling, cv, test_split, first_trained_module):
  series_fmt = {phases_friendly_names[phase].lower():(f'{phase}_' + '{}') for phase in phase_data_dirs_resized.keys()}
  live_loss_plot = PlotLosses(series_fmt=series_fmt,
                     #mark_high_score={'valid_ROC AUC': 'higher'},
                     validation_fmt=None)

  model = model.to(device)

  save_path = init_log(model.__class__.__name__)

  starting_time = time.time()

  best_model_wts = copy.deepcopy(model.state_dict())  

  # phase_y = {phase for (phase) in phase_data_dirs_resized.keys()}
  phase_y = {phase: get_true_labels(phase_data_dirs_resized[phase]) for (phase) in ['train', 'validation']}
  split_strategy = get_split_strategy(phase_y, num_epochs, cv, test_split)  
  
  first_trained_module_i, first_trained_module, optimizer = find_first_trained_module(model, start_module_name=first_trained_module)  
  log_model_summary(model, criterion, optimizer, phase_data_dirs_resized, over_sampling, cv, True)
  
  return model, optimizer, first_trained_module_i, save_path, starting_time, best_model_wts, phase_y, split_strategy, live_loss_plot

time: 1.34 ms (started: 2023-04-22 14:31:08 +00:00)


In [58]:
phases_friendly_names = {'train': 'Training', 'validation': 'Validation', 'test': 'Test'}

time: 553 µs (started: 2023-04-22 14:31:08 +00:00)


I extracted functions to initialize, run and terminate a single epoch. So that it can be reused by test_model later.

In [59]:
def init_epoch(model, phase, folders, indices=None, phase_y=None, over_sampling=False, epoch=None, num_epochs=1, cv=None, is_test_phase=False, n_crops=None, n_crops_stages=None, best_auc=1, resize=0.75):
  
  if n_crops is None:
    n_crops = get_n_crops(is_test_phase, best_auc, n_crops_stages)
  
  fold = '' if cv is None else f'- KFold {cv if (epoch % cv) == 0 else (epoch % cv)}/{cv} '
  crops = '' if phase=='train' else f'{n_crops}-crop' + ('' if n_crops==1 else 's')
  epoch = '' if num_epochs==1 else f'Epoch {epoch}/{num_epochs} '
  logging.info(f'\n{epoch}{fold}{phases_friendly_names[phase]} {crops}')
  
  return get_data_loader(phase, model, folders, indices, phase_y, over_sampling, best_auc, n_crops, resize)

time: 5.01 ms (started: 2023-04-22 14:31:08 +00:00)


In [60]:
def init_epoch_variables():

  running_loss = 0.0
  
  # {disease: = [nb corrects, nb processed]}
  running_accuracy = {disease:[0, 0] for disease in classes}
 
  # true one hot labels
  one_hot_labels = np.empty((0,3), int)
  
  # predicted outputs probabilities
  outputs_probabilities = np.empty((0,3), float)
  
  return running_loss, running_accuracy, one_hot_labels, outputs_probabilities

time: 1.01 ms (started: 2023-04-22 14:31:08 +00:00)


In [94]:
def run_epoch(phase, data_loader, model, criterion, optimizer=None, logs={}, best_auc=0.0, best_model_wts=None, save_path=None, save_phase='test'):
  # train, validate or test the model for one epoch
  
  if phase == 'train':
      model.train()  # Set model to training mode
  else:
      model.eval()   # Set model to evaluate mode
      
  # reset epoch metrics
  running_loss, running_accuracy, one_hot_labels, outputs_probabilities = init_epoch_variables()

  # Iterate over data.
  tqdm_items = tqdm(data_loader)

  train_labels = next(iter(data_loader))

  for x in tqdm_items:
      inputs = inputs.to(device)
      labels = labels.to(device)


      if optimizer is not None:
        # zero the parameter gradients
        optimizer.zero_grad()

      # track history if only in train
      with torch.set_grad_enabled(phase == 'train'):
          # forward
          outputs, loss = batch_forward(inputs, labels, model, model.__class__.__name__, criterion, phase)

          # backward + optimize only if in training phase
          if phase == 'train':
              loss.backward()
              optimizer.step()

          # compute and accumulate metrics          
          running_loss, one_hot_labels, outputs_probabilities, running_accuracy \
            = compute_batch_metrics(inputs, labels, outputs, loss, running_loss, one_hot_labels, outputs_probabilities, running_accuracy)
          
          # update progress bar status
          tqdm_items.set_postfix_str(get_running_status(phase, running_loss, running_accuracy))

  epoch_loss = running_loss / len(data_loader.sampler)
  epoch_acc = compute_accuracy(running_accuracy)
  epoch_auc = get_roc_auc(one_hot_labels[:,[0,2]], outputs_probabilities[:,[0,2]])
  
  logging.info('Items processed: {}'.format([f'{disease}: {running_accuracy[disease][1]}' for disease in running_accuracy]))
  logging.info(get_running_status(phase, running_loss, running_accuracy))
  logging.info(f'Cat 1 ROC AUC: {epoch_auc[0]:.3f} Cat 2 ROC AUC: {epoch_auc[1]:.3f} Cat 3 ROC AUC: {epoch_auc[2]:.3f}')

  logs = update_liveplot(logs, phase, epoch_loss, epoch_acc, epoch_auc)

  # deep copy the model
  if phase == save_phase and epoch_auc[2] >= best_auc:
      best_model_wts, best_auc = save_model(save_path, model, epoch_auc[2])
      
  return logs, best_auc, best_model_wts, running_accuracy, outputs_probabilities

time: 17.6 ms (started: 2023-04-22 16:07:21 +00:00)


In [62]:
def compute_accuracy(running_accuracy):
  
  running_accuracy_array = np.array(list(running_accuracy.values()))
  
  return sum(running_accuracy_array[:,0]) / sum(running_accuracy_array[:,1])

time: 5.33 ms (started: 2023-04-22 14:31:08 +00:00)


In [63]:
def terminate_epoch(live_loss_plot, logs):
  
  live_loss_plot.update(logs)
  live_loss_plot.draw()

  logging.info('')

time: 694 µs (started: 2023-04-22 14:31:08 +00:00)


I also extracted a function to calculate outputs and loss for a single batch.  

Note there are two special cases:
- special case for [Inception3](https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958) while training only because it has an auxiliary output;
- special case for multi-crop as tensor must be transformed from 5D to 4D for computation.

## Ready? GO!

And now let's train the model with 5-fold cross validation.  
Note that normally, I should have used and plotted mean validation metric every 5 epochs. But that's not what I did.

If you want to train your own model, just uncomment the code below, run it, and take a break... a multiple hours break 😴 💤...  

Or keep an eye on my addictive progress bar with live loss and per-class accuracy 😵😃

In [102]:
from tqdm import tqdm_notebook as tqdm
def train_densenet_model():

  model, image_size = initialize_model('densenet', 2, use_pretrained=True)

  return train_model(model, criterion,phase_data_dirs_resized, over_sampling=True, cv=5, first_trained_module='features.denseblock3', num_epochs=100)

# uncomment the code below to train it from scratch...
dr_densenet_model = train_densenet_model()

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet161_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet161_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Start model init


Start training DenseNet model


Length of phase_y train: 75920


  Class distribution (dict_values([62128, 11978, 1314, 300, 104, 78, 14, 2, 2])): 62128 unknown, 11978 nevus, 1314 melanoma, 300 seborrheic keratosis, 104 lentigo NOS, 78 lichenoid keratosis, 14 solar lentigo, 2 cafe-au-lait macule, 2 atypical melanocytic proliferation
  Oversampling: True
  Cross validation: 5
  Loss function: CrossEntropyLoss() - Weights: tensor([1.5000, 1.0000, 1.0000])
  Optimizer: Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    group_name: features.denseblock3
    lr: 5e-05
    maximize: False
    weight_decay: 1e-06

Parameter Group 1
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    group_name: features.transition3
    lr: 0.0001
    maximize: False
    weight_decay: 1e-06

Parameter Group 2
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: F

peeeen
dict_items([(0.0, 62128), (1.0, 11978), (2.0, 1314), (3.0, 300), (4.0, 104), (5.0, 78), (6.0, 14), (7.0, 2), (8.0, 2)])
Done


/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/mod

{'train': ['/content/drive/MyDrive/trained_resized/ISIC_2637011.jpg', '/content/drive/MyDrive/trained_resized/ISIC_0015719.jpg', '/content/drive/MyDrive/trained_resized/ISIC_0052212.jpg', '/content/drive/MyDrive/trained_resized/ISIC_0068279.jpg', '/content/drive/MyDrive/trained_resized/ISIC_0074268.jpg', '/content/drive/MyDrive/trained_resized/ISIC_0074311.jpg', '/content/drive/MyDrive/trained_resized/ISIC_0074542.jpg', '/content/drive/MyDrive/trained_resized/ISIC_0075663.jpg', '/content/drive/MyDrive/trained_resized/ISIC_0075914.jpg', '/content/drive/MyDrive/trained_resized/ISIC_0076262.jpg', '/content/drive/MyDrive/trained_resized/ISIC_0076545.jpg', '/content/drive/MyDrive/trained_resized/ISIC_0076742.jpg', '/content/drive/MyDrive/trained_resized/ISIC_0076995.jpg', '/content/drive/MyDrive/trained_resized/ISIC_0077472.jpg', '/content/drive/MyDrive/trained_resized/ISIC_0077735.jpg', '/content/drive/MyDrive/trained_resized/ISIC_0078703.jpg', '/content/drive/MyDrive/trained_resized/ISIC_

UnboundLocalError: ignored

time: 4min 5s (started: 2023-04-22 16:57:43 +00:00)


![title](https://github.com/sebastienlange/dermatologist-ai/blob/master/images/densenet_training_plots.png?raw=1)



```
Epoch 70/100 - KFold 5/5
log loss:
training   (min:    0.057, max:    0.679, cur:    0.062)
validation (min:    0.029, max:    0.698, cur:    0.040)

accuracy:
training   (min:    0.685, max:    0.979, cur:    0.978)
validation (min:    0.741, max:    0.991, cur:    0.983)

ROC AUC:

training   (min:    0.885, max:    0.999, cur:    0.999)
validation (min:    0.918, max:    1.000, cur:    1.000)

Items processed: {'melanoma': 1510, 'nevus': 1510, 'seborrheic_keratosis': 1510}
train_loss=0.052, train_acc=0.981, melan_acc=0.979, nevus_acc=0.966, sebor_acc=0.997
Cat 1 ROC AUC: 0.998 Cat 2 ROC AUC: 1.000 Cat 3 ROC AUC: 0.999

Items processed: {'melanoma': 377, 'nevus': 377, 'seborrheic_keratosis': 377}
valid_loss=0.059, valid_acc=0.984, melan_acc=0.958, nevus_acc=0.995, sebor_acc=1.000
Cat 1 ROC AUC: 0.999 Cat 2 ROC AUC: 1.000 Cat 3 ROC AUC: 1.000
```

`

While validation metrics are excellent after 70 epochs, test metrics have more variations ! It overfits to the training/validation set. So I did not keep the very last model. But one of the previous.  

I should also mention that, in the run above, I did not add too much blur, noise and color jitter in train transforms. Hence the validation metrics that reach 1.000 ROC AUC and 99.1% accuracy.  

I kept these (nice) plots because it was one of my longest runs, but the model I kept is from a shorter run, with the data augmentation defined earlier in this book.

# Evaluate the Model

Inspired by the [2017 ISIC Challenge on Skin Lesion Analysis Towards Melanoma Detection](https://challenge.kitware.com/#challenge/583f126bcad3a51cc66c8d9a), the algorithm is ranked according to three separate categories:
- Category 1: ROC AUC for Melanoma Classification
- Category 2: ROC AUC for Melanocytic Classification
- Category 3: Mean ROC AUC

The test_model function below calculates the test Mean ROC AUC of the model on the test set.  
It also returns a data frame with the probabilities of detection of melanoma and seborrheic keratoses in the given images.  

In [166]:
def test_model(images_folder, criterion, model=None, resize=0.75, n_crops=10, plot=True, model_fn=None, force_test=False):
  
  predictions = None
  if model_fn is not None and not force_test:
    predictions, accuracy, best_auc = load_results(model_fn, n_crops, plot)
  
  if predictions is None:  
    preprocess_images(images_folder)

    if model is None: model = load_model(model_fn)

    # make batch_size inversely proportional to the crop and compatible with biggest models... otherwise CUDA out of memory ;-)
    batch_size['test'] = int(40 / n_crops)

    if len(logging.getLogger().handlers)==1:
      init_log('test')

    data_loader = init_epoch(model, 'test', [g_train_path], n_crops=n_crops)

    _, best_auc, _, accuracy, data = run_epoch('test', data_loader, model, criterion)

    file_names = [os.path.basename(fn) for fn in sorted(glob.glob(os.path.join(g_train_path, '*', '*.jpg')))]
    predictions = pd.DataFrame(index=file_names, data=data, columns=classes)

    if plot:
      _ = get_roc_auc(y_true, predictions.as_matrix(columns=[classes[0], classes[2]]), plot=True)

    save_results(model_fn, n_crops, predictions)
  
  return predictions, accuracy, best_auc, model

plt.rcParams['figure.figsize'] = (8, 6)

time: 8.51 ms (started: 2023-04-22 12:26:08 +00:00)


Thanks to functions below, test_model also save results to a csv file so that the heavy computation is done once for all... For any subsequent tests with the same model filename, the results are simply reloaded. Use force_test=True to override this behavior.

In [ ]:
def save_results(model_fn, n_crops, predictions):
  # test once for all... it takes time :-)
  
  path_to_save = os.path.join(google_drive_shared_path if os.path.exists(google_drive_shared_path) else os.getcwd(), 'results')
  predictions.to_csv(os.path.join(path_to_save, model_fn.replace('.pt', f'-{n_crops}-crops.csv'))) 
  
def load_results(model_fn, n_crops, plot):
  
  predictions = None
  accuracy = None
  best_auc = None
  results_fn = model_fn.replace('.pt', f'-{n_crops}-crops.csv')

  if os.path.exists(os.path.join(os.getcwd(), 'results', results_fn)):
    results_fn = os.path.join(os.getcwd(), 'results', results_fn)
  elif os.path.exists(os.path.join(google_drive_shared_path, 'results', results_fn)):
    results_fn = os.path.join(google_drive_shared_path, 'results', results_fn)
  else:
    download('https://raw.githubusercontent.com/sebastienlange/dermatologist-ai/master/results/' + results_fn, os.path.join(os.getcwd(), 'results'))
    results_fn = os.path.join(os.getcwd(), 'results', results_fn)

  if os.path.exists(results_fn):
    if plot: print(f'Loading results from {results_fn}...\n')
    predictions = pd.read_csv(results_fn)
    best_auc, accuracy = compute_scores(predictions.as_matrix(columns=[classes[0], classes[1], classes[2]]), plot)
    if plot:
      print(f'        Accuracy: {accuracy:.3f}')
    else:
      print(f'Loaded results from {results_fn}: accuracy = {accuracy:.3f} - ROC AUC Cat3 = {best_auc:0.3f}')
             
  return predictions, accuracy, best_auc
             
def compute_scores(predictions, plot=False):

  combined_roc_auc = get_roc_auc(y_true, predictions[:, [0,2]], plot=plot)[2]
  combined_accuracy = sum(y == np.argmax(predictions, axis=1))/predictions.shape[0]
  
  return combined_roc_auc, combined_accuracy

I need to download the ground_truth.csv where true labels are stored for melanoma and seborrheic keratoses.

In [ ]:
download('https://raw.githubusercontent.com/sebastienlange/dermatologist-ai/master/ground_truth.csv')

In [ ]:
def get_ground_truth():
    # get ground truth labels for test dataset
    truth = pd.read_csv('ground_truth.csv')
    y_true = truth.as_matrix(columns=["task_1", "task_2"])
    return y_true
  
y_true = get_ground_truth()
y = np.argmax([[0, 1, 0] if sum(arr)==0 else arr for arr in np.insert(y_true, 1, 0, axis=1)], axis=1)

Now we are ready to __evaluate__ our pretrained DenseNet model, plot __ROC AUC__ and see how it performs! Let's name it __Dr DenseNet Model__. 

In [167]:
# uncomment force_test=True if you want to skip loading results but rather force evaluating the model
predictions, accuracy, best_auc, dr_densenet_model = test_model(g_train_path, criterion, model_fn='DenseNet.pt',
                                                               #force_test=True
                                                               )

NameError: ignored

time: 32.2 ms (started: 2023-04-22 12:26:13 +00:00)


__0.914__! Well done!!!

__Dr. DenseNet__ is an excellent Doctor, with excellent results!  If he had participated in the [2017 ISIC Challenge on Skin Lesion Analysis Towards Melanoma Detection](https://challenge.kitware.com/#challenge/583f126bcad3a51cc66c8d9a), he __would have reached the TOP 1__ with a 0.003 gain over the highest score!

📝But it's still a human and sometimes he makes mistakes. As I am a rather anxious person, I would prefer to __ask for a second medical opinion__... By far, my biggest learning experience was: do not be too confident in one single doctor 😉 📝

In [ ]:
release_model(dr_densenet_model)

# Go Further

## Second Medical Opinion

So let's seek for a second medical opinion from __Dr. [Inception v3](https://arxiv.org/abs/1512.00567)__.  

![Inception v3](https://cdn-images-1.medium.com/max/960/1*gqKM5V-uo2sMFFPDS84yJw.png)

Note that I again need to specify the layers I want to freeze (feature extraction) and the ones I want to train (fine-tuning). I choose to fine-tune from the layer named Mixed_5c.

In [ ]:
def train_inception3_model():
  
  model, image_size = initialize_model('inception3', len(classes), use_pretrained=True)

  batch_size['train'] = 128
  
  return train_model(model, criterion, phase_data_dirs_resized, over_sampling=False, cv=5,
                     first_trained_module='Mixed_5c', num_epochs=100, fine_tuning_module_rounds=2)

# uncomment the code below to train it from scratch...
# dr_inception_model = train_inception3_model()

In [ ]:
# load my highest Inception3 scores
predictions, accuracy, best_auc, dr_inception_model = test_model(data_dir['test'] + '_resized', criterion, model_fn='Inception3_3_0.9089.pt')

Same score as the winner of the challenge for Dr Inception!

In [ ]:
release_model(dr_inception_model)

## Third Medical Opinion

I now have two medical opinions! Great! But __Dr. DenseNet and Dr. Inception sometimes gives me opposite opinion!__  
So let's look for a third medical opinion that will be able to decide: __Dr. [NASNetALarge](https://arxiv.org/abs/1707.07012) arrives on stage!__  

As stated in the [paper](https://arxiv.org/abs/1707.07012) that describes it, this is the __most accurate architecture__ as of April 2018:

![NasNet performance](https://1.bp.blogspot.com/-E1qM-CKq-BA/WfuGc22fPBI/AAAAAAAACIg/frpwbO5Jh-oL0cSObyJa29fXkBsuVl7CACLcBGAs/s640/image3.jpg)

Again I need to specify the layers I want to fine-tune. I "arbitrarily" choose to set the fine-tuning cursor at a layer named cell_4.  

Ok it is the most accurate, but as the name suggests: it is large, very large. I'm not sure I'm patient enough to train it successfully, especially since I made an arbitrary choice.

And last thing ... Google Colab __lacks GPU memory__ to train it worthily! I have to __reduce the train batch size to 12__.

In [ ]:
batch_size = {phase: 12 if phase=='train' else 2 for phase in phases}

def train_nasnetalarge_model():
  
  model, image_size = initialize_model('nasnetalarge', len(classes), use_pretrained=True)

  return train_model(model, criterion, phase_data_dirs_resized, over_sampling=True, cv=5,
                     first_trained_module='cell_4', num_epochs=100)

# uncomment the code below to train it from scratch...
# dr_nasnetalarge_model = train_nasnetalarge_model()

I commented the code below because initializing NASNetALarge takes more time. But feel free to uncomment it!

In [ ]:
predictions, accuracy, best_auc, dr_nasnetalarge_model = test_model(data_dir['test'] + '_resized', criterion, model_fn='NASNetALarge_4_0.9106.pt')

0.914! High score for Dr NasNetALarge!

In [ ]:
release_model(dr_nasnetalarge_model) 

## Just another one for fun

Hey, I can not help but seek __Dr. Xception's__ medical opinion! 😁

In [ ]:
def train_xception_model(model=None):

  model, image_size = initialize_model('xception', len(classes), use_pretrained=True)

  return train_model(model, criterion, phase_data_dirs_resized, over_sampling=True, cv=5,
                     first_trained_module='block4', num_epochs=100, fine_tuning_module_rounds=5, 
                     n_crops_stages={5: 0.885, 10: 0.900})

# uncomment the code below to train it from scratch...
#dr_xception_model = train_xception_model()

In [ ]:
# load my highest Xception score
predictions, accuracy, best_auc, dr_xception_model = test_model(data_dir['test'] + '_resized', criterion, n_crops=10, model_fn='Xception_2_block3.rep.4.conv1_0.9185.pt')

0.918! Highest score for Dr Xception!

In [ ]:
release_model(dr_xception_model) 

## Committee of Doctors

I'm lucky enough to bring together my four doctors in a committee! Alone, they all fit in the TOP 1! What a dream team! 😲

They share their results, debate, and __Dr. Inception comes to suggest looking at the dermatoscopic images from different perspectives__.  

He suggest the following crops:

In [ ]:
crops = [1, 5, 10, 20]

But wait a minute ... actually I just had a __hard time stopping to learn 📝 and try other ideas and other models ...__ 😅  

In [ ]:
#@title
def get_unique_models(model_names):
  
  return sorted(set(model_names))

def get_unique_models_count(model_names):
  
  return len(get_unique_models(model_names))

trained_models = list(trained_models_files.keys())
model_names = [get_model_name(fn) for fn in trained_models_files.keys()]

HTML(f"In the end, I kept {len(trained_models)} trials from <b>{get_unique_models_count(model_names)} unique models</b> {get_unique_models(model_names)}, each tested at {len(crops)} different crops {crops}, this makes a total of <b>{len(trained_models)*len(crops)} different perspectives</b>. It's too much.<p>Lets' choose some <b>threshold to eliminate the worst ones</b>:")

In [ ]:
def threshold_results(roc_auc_value, accuracy):
  
  accuracy = accuracy if type(accuracy)==float or type(accuracy)==np.float64 else compute_accuracy(accuracy)
  
  return (round(roc_auc_value,3) >= 0.907 and round(accuracy,2)>=0.75) \
       or round(roc_auc_value,3) > 0.911 \
       or round(accuracy,2) >= 0.80

In [ ]:
#@title
def test_model_with(model_fn, folder, crops, criterion):
  # test model once per given crop
  
  all_predictions = {}
  model = None
  
  for crop in crops:
      
    print(f'\nTesting {model_fn} with {crop}-crops')
    predictions, accuracy, best_auc, model = test_model(folder, criterion, model, plot=False, n_crops=crop, model_fn=model_fn)     
    all_predictions[crop] = (predictions, accuracy, best_auc)
    
  if model is not None:
    release_model(model)
      
  return all_predictions

HTML(f"Now I load my <b>{len(trained_models)} trials</b>... and test them at {len(crops)} different crops {crops}.")

As the test results of the trained models are available on my github account, "test" represents only the cost necessary to download .csv files and to evaluate parameters ... otherwise it takes several hours!

In [ ]:
#@title
# test all models with given crops
predictions = {i:test_model_with(model_fn, data_dir['test'] + '_resized', crops, criterion) for i, model_fn in enumerate(trained_models)}

### Single-model scores

Here are their individual scores with different crops:

In [ ]:
#@title
def get_summary_row(test_result, n_crop, model_name):
    
  return [f'Dr. {model_name}', 
          n_crop, 
          np.round(test_result[1] if type(test_result[1])==float or type(test_result[1])==np.float64 else compute_accuracy(test_result[1]), 3), 
          np.round(test_result[2], 3)]

def append_summary_row(summary, new_row, background=''):
  
  return summary.append({'Model': f'<b>{new_row[0]}</b>', 
                'Crop':f'{new_row[1]}', 
                'Accuracy': '<b{0}>{1:.3f}</b>'.format(background, new_row[2]),
                'Cat 3 ROC AUC': '<b{0}>{1:.3f}</b>'.format(background, new_row[3])
                         }, ignore_index=True)

def create_summary(predictions, trained_model_names, crops):
  
  data = [get_summary_row(predictions[model_idx][crop], crop, trained_model_names[model_idx]) for crop in crops for model_idx in range(len(trained_models))
         if threshold_results(predictions[model_idx][crop][2], predictions[model_idx][crop][1])]
  summary = pd.DataFrame(data=data, columns=['Model', 'Crop', 'Accuracy', 'Cat 3 ROC AUC'])
  summary = summary.sort_values(by=['Cat 3 ROC AUC', 'Accuracy', 'Model'], ascending=[False, False, True])
  summary = append_summary_row(summary, ['Mean', '', summary['Accuracy'].mean(), summary['Cat 3 ROC AUC'].mean()])
  
  return summary

summary = create_summary(predictions, trained_models, crops)
HTML(summary.to_html(escape=False))

In [ ]:
#@title
mean_auc = summary.iloc[-1]['Cat 3 ROC AUC']
min_auc = np.min([item for item in summary['Cat 3 ROC AUC'][:-1]])
max_auc = np.max([item for item in summary['Cat 3 ROC AUC'][:-1]])
HTML(f'So the chosen threshold makes it possible to select the {len(summary)-1} best persepectives out of {len(trained_models)*len(crops)} available.<p>Their individual Cat 3 ROC AUC range from <b>{min_auc} to {max_auc}</b>, with an <b>average of {mean_auc}</b>.')

### Build the best team

At this stage, my first feeling was to try a __weighted average__ to give more importance to my prefered (and most trained) models 😉. But it was __too complex__ for a first attempt and not necessary at all!

📝 Indeed, a __simple average works just fine__! So let's give equal value to each perspective and extract for each image the mean of all probabilities.📝  

However, I do not average all the trials, but I try many n-combinations from unique architectures: I __build the best team__ to achieve the __highest metrics__ while giving each architecture __at most a unique and equal voting right__.

In [ ]:
#@title
import itertools

def get_combined_predictions(predictions_by_model, combination):
  
  sub_predictions_by_model = {(model_idx, crop):predictions_by_model[(model_idx, crop)] for (model_idx,crop) in combination }
  
  # for every image, I calculate the mean probability of my combinations
  return np.mean(list(sub_predictions_by_model.values()), axis=0)

def step(x, threshold=0):
    return np.sign(np.sign(x-threshold)+1)

def minimum_cost_threshold(y_true, y_pred, costs=(1, 1)):
    '''
    Find the decision threshold that minimizes the cost of errors.
    
    costs=(false_positive_cost, false_negative_cost)
    '''
    def error(x, costs):
        return costs[0]*step(-x, 0.99) + costs[1]*step(x,0.99)
    costs = np.array(costs)
    costs = costs/np.sum(costs)
    min_cost = np.inf
    min_threshold = 0
    for threshold in np.arange(0, 1, 0.001):
        y_pred_bin = np.ceil(y_pred - threshold)
#         cost = np.abs(y_true - y_pred_bin)*np.apply_along_axis(lambda i: costs[i[0]], 0, y_true)
        cost = error(y_true-y_pred_bin, costs)
        if np.mean(cost) < min_cost:
#             print(cost[:10])
            min_cost = np.mean(cost)
            min_threshold = threshold
    return min_threshold
  
def get_models_and_crops(combination):
  return sorted([f'{trained_models[model_idx]} ({crop}-crops)' for (model_idx,crop) in combination])
  
def get_predictions_list(combination):
  return list({(model_idx, crop):predictions_by_model[(model_idx, crop)] for (model_idx,crop) in combination }.values())
  
def process_combination(combination, very_best_auc, best_melanoma_f1_threshold, best_seborreric_f1_threshold, very_best_comb):
  
  # keep only combinations with unique architecture
  if get_unique_models_count([get_model_name(trained_models[i]) for (i,crop) in combination]) == len(combination):
    
    combined_predictions = get_combined_predictions(predictions_by_model, combination)
    combined_roc_auc, combined_accuracy = compute_scores(combined_predictions)
    
    auc_data.append({'ROC AUC': combined_roc_auc, 'multi-model': n, 'type': 'mean', 'crop': (combination[0][1] if len(combination)==1 else '')})        

    seq = [x['ROC AUC'] for x in auc_data if x['multi-model']==n]
    current_min = np.min(seq)
    current_max = np.max(seq) 

    if combined_roc_auc == current_max or combined_roc_auc == current_min:
      tqdm_items.set_postfix_str(f'ROC AUC min.-max.: {current_min:0.3f}-{current_max:0.3f}')
      
    # scores are sorted by roc_auc, melanoma_f1_threshold, seborreric_f1_threshold
    round_combined_roc_auc = round(combined_roc_auc,3)
    if round_combined_roc_auc >= very_best_auc:

      melanoma_f1_threshold = round(minimum_cost_threshold(y_true[:, 0], combined_predictions[:, 0], costs=(1,10)),3)
      if round_combined_roc_auc > very_best_auc or melanoma_f1_threshold >= best_melanoma_f1_threshold:
        seborreric_f1_threshold = round(minimum_cost_threshold(y_true[:, 1], combined_predictions[:, 2], costs=(1,10)),3)
        if melanoma_f1_threshold > best_melanoma_f1_threshold or seborreric_f1_threshold >= seborreric_f1_threshold:

          if very_best_auc == 0:

            print('High scores')
            print('------|---------|---------|----------|--------------------------------------------------------------------------------------------------')
            print(' ROC  | F1 mel. | F1 seb. | Accuracy | Models (n-crops)')
            print(' AUC  | thresh. | thresh. |          |')
            print('------|---------|---------|----------|--------------------------------------------------------------------------------------------------')

          very_best_auc = round_combined_roc_auc
          best_melanoma_f1_threshold = melanoma_f1_threshold
          best_seborreric_f1_threshold = seborreric_f1_threshold
          very_best_comb = combination
          
          if len(combination) > 1:
            models_and_crops = get_models_and_crops(combination)
            models_and_crops = ' + '.join(models_and_crops)
            print(f'{combined_roc_auc:.3f} |  {best_melanoma_f1_threshold:.3f}  |  {best_seborreric_f1_threshold:.3f}  |  {combined_accuracy:.3f}   | {models_and_crops}')
             
  return very_best_auc, best_melanoma_f1_threshold, best_seborreric_f1_threshold, very_best_comb  


print('Analyzing combinations of models...')

predictions_by_model = {(model_idx, crop): predictions[model_idx][crop][0].as_matrix(columns=[classes[0], classes[1], classes[2]]) 
                        for crop in crops for model_idx in range(len(trained_models))
                        if threshold_results(predictions[model_idx][crop][2], predictions[model_idx][crop][1])}

auc_data = []

auc_meaning = [classes[0], classes[2], 'mean']
very_best_auc = 0
very_best_melanoma_f1_threshold = 0
very_best_seborreric_f1_threshold = 0
very_best_comb = ()

unique_fn = set([item.replace('Dr. ', '') for item in summary['Model'][:-1]])
n_unique_architectures = get_unique_models_count([get_model_name(fn) for fn in unique_fn])

try:
  for n in np.arange(1, n_unique_architectures+1):      
    print('\n' + f'Combining {n} models out of {len(predictions_by_model)} from {n_unique_architectures} unique architectures...')

    best_melanoma_f1_threshold = 0
    best_seborreric_f1_threshold = 0
    
    combinations = list(itertools.combinations(predictions_by_model.keys(), n))
    tqdm_items = tqdm(combinations)

    for combination in tqdm_items:
      very_best_auc, best_melanoma_f1_threshold, best_seborreric_f1_threshold, very_best_comb = process_combination(combination, very_best_auc, best_melanoma_f1_threshold, best_seborreric_f1_threshold, very_best_comb) 

except KeyboardInterrupt:
    pass    


### Multi-model scores

In [ ]:
#@title
def get_crops(m):
  return m.split('(')[1].split('-')[0]

def get_fn(m):
  return m.split('(')[0].strip()

def get_score(fn, crop):
  return summary[(summary['Model']==f'Dr. {fn}') & (summary['Crop']==int(crop))].iloc[0]['Cat 3 ROC AUC']

def print_team(name, note, combination, score):
  return f'The <b>{name} reaches a score of {score:0.3f}</b> and consists of:<p><i>{note}</i><p><ol><li>' + '<li>'.join([f'<b>Dr. {get_model_name(get_fn(m))}</b> ({get_crops(m)}-crops) with an individual score of <b>{get_score(get_fn(m), get_crops(m))}</b> from trial {get_fn(m)}</li>' for m in sorted(get_models_and_crops(combination))]) + '</ol>'
  
HTML(print_team('very best team(*)', '(*) based on ROC AUC score only', very_best_comb, very_best_auc))

In [ ]:
#@title
print('Very best ROC AUC scores:')
combined_predictions = get_combined_predictions(predictions_by_model, very_best_comb)
combined_roc_auc, combined_accuracy = compute_scores(combined_predictions, True)

Wow!  Great!  

__Mean ROC AUC score climbs to 0.944 and outperforms the first score of the initial challenge!__

Editor's note: before giving a try to multiple models, I had, a long time ago, written in my conclusions: *"I only tried DenseNet with all parameters freezed! I'm pretty sure that a combination of Resnet / Inception / VGG / DenseNet with some fine-tuned layers and a [VotingClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingClassifier. html) would push the limits higher!"*. And I was pretty right! 😃😃😃

Indeed while I had the intuition of training multiple models early in this project, I discovered afterwards (see Comparison with State-of-the-art Approaches in [Review: Inception-v4](https://towardsdatascience.com/review-inception-v4-evolved-from-googlenet-merged-with-resnet-idea-image-classification-5e8c339d18bc)) that this was a __best practice__ called __multi-model__. And this is at the same time I discovered and implemented __multi-crop__ in test while I already had implemented __multi-scale__ but I did not keep it.

However, IMHO this "very best" team requires a minimum cost threshold that is too low for an __optimized confusion matrix__. So in __real world__, to reach a __better compromise between ROC AUC and min. cost threshold__, I would __select the team below__ while admitting his minimum cost remains too low for what I expected:

In [ ]:
#@title
selected_combination = ((trained_models.index('Inception3_3_0.9051.pt'),10),
                      (trained_models.index('NASNetALarge.pt'),10),
                      (trained_models.index('SENet_3_layer2.1_0.8988.pt'),10),
                      (trained_models.index('Xception_1_block4_0.9120.pt'),5))

selected_predictions = get_combined_predictions(predictions_by_model, selected_combination)
selected_roc_auc, selected_accuracy = compute_scores(selected_predictions, False)

HTML(print_team('selected best team(*)', '(*) based on a good compromise between ROC AUC and min. cost threshold for an optimized confusion matrix', selected_combination, selected_roc_auc))

In [ ]:
#@title
print('Multi-model selected with a good compromise between ROC AUC and min. cost threshold for an optimized confusion matrix:')

selected_roc_auc, selected_accuracy = compute_scores(selected_predictions, True)

In [ ]:
#@title
download('https://raw.githubusercontent.com/sebastienlange/dermatologist-ai/master/get_results.py')

from get_results import plot_confusion_matrix

def plot_best_confusion_matrix(skin_disease, y_true, y_pred, costs=(1,1)):
    threshold = minimum_cost_threshold(y_true, y_pred, costs=costs)
    print(f'Best threshold for {skin_disease}: {threshold:0.2f}')
    plot_confusion_matrix(y_true, y_pred, threshold, ['benign', 'malignant'])
    
plot_best_confusion_matrix(classes[0], y_true[:, 0], selected_predictions[:, 0], costs=(1,10))

In [ ]:
#@title
plot_best_confusion_matrix(classes[2], y_true[:, 1], selected_predictions[:, 2], costs=(1,10))

### Multi-model statistics

Before concluding, I also found very interesting to make some statisticts on multi-crop / multi-model scores.

A picture is worth a thousand words! Let's draw a box plot to show __distributions of ROC AUC with respect to number of models__:

In [ ]:
#@title
best_scores = [{'n-model': 1, 'score': summary[summary['Crop']==1].iloc[0]['Cat 3 ROC AUC'], 'description': 'my best single-crop / single-model score'},
               {'score': 0.911, 'description': 'challenge win score'},
               {'n-model': 1, 'score': summary.loc[0]['Cat 3 ROC AUC'], 'description': 'my best multi-crop / single-model score'},
               {'n-model': 5, 'score': 0.938, 'description': 'my best single-crop / multi-model score'},
               {'n-model': len(selected_combination), 'score': selected_roc_auc, 'description': 'my prefered multi-model (compromise betw. ROC AUC and min. cost threshold)'},
               {'n-model': 6, 'score': combined_roc_auc, 'description': 'my best multi-crop / multi-model score'},
              ]

auc_df = pd.DataFrame(columns=['ROC AUC', 'multi-model', 'type'], data=auc_data)

sns.set(style="whitegrid", palette="pastel", color_codes=True)
ax = sns.boxplot(x="multi-model", y="ROC AUC", data=auc_df)
ax.set_title('Min/max scores by multi-models with outliers')

for best_score in best_scores:
  high_score = best_score['score']
  ax.axhline(high_score, ls='--')
  annotation = ax.text(n_unique_architectures - 0.4, high_score, f'{high_score:0.3f}: ' + best_score['description'])
  if 'n-model' in best_score:
    plt.plot(best_score['n-model']-1, high_score, 'y*', markersize=15, markeredgewidth=2, markeredgecolor='k')

This is interesting to see that <b>multi-model gives me ≃3.25% return over investment</b>... whereas multi-crop "only" gives ≃0.6%. But still that adds up 😃  

# Conclusions

I was able to achieve a __Mean ROC AUC score of 0.944__. It would have been a __TOP 1__ in the initial challenge (see scores below). It's very satisfying for what I wanted to achieve, especially since the __winner's score is 0.911__.  😃

But much more than this score, I learned a lot 📝📝📝, sometimes the hard way 😓, and took a lot of fun. 😅  

From the first day I reached TOP 3, I said to myself: _"let's publish tomorrow and continue your [Deep Learning nanodegree](https://www.udacity.com/course/deep-learning-nanodegree--nd101). This is an optional mini-project and you're now in a late for the rest of the nanodegree"_... but I just wasn't able to stop training and learning by myself. It took me six more weeks of learning 📝📝📝📝 and wonderful experience to go from 0.900 to 0.944 ✨. Now I have to catch up on my nanodegree! 

![title](https://github.com/sebastienlange/dermatologist-ai/blob/master/images/cat_3.png?raw=1)

## Possible points of improvement
Yes! I think it's possible to further improve the results and I still have a lot to learn! 

So here are possible points of improvement:
- test loss is often much higher than training/validation loss; although it was not the measure observed during the challenge, I observed it and felt that it was showing too much overfitting to the training/validation set (same for accuracy but to a lesser extent). I need __more data__! Moreover, as already said, the minimum cost threashold for an optimized confusion matrix remains too low for what I expected;
- more or different __data augmentations__ techniques. Particularly, I integrated [imgaug](https://github.com/aleju/imgaug) lately, so there's __a lot to try with [imgaug](https://github.com/aleju/imgaug)__! Particularly add more noise in image (like improved fake hairs, patch and add fake ruler)
- I chose to divide more or less in two equal parts, (or at one third) the number of feature extracted and fine-tuned layers. So, __I'm pretty sure there's a better point than "randomly" choosing first half of layers as feature extracted and second half as fine-tuned layers__. Hence my late try to add one more fine-tuned layer to train every n epochs rounds without metrics improvement; 
- use different source of images : all the images come from the same distribution
- try [VotingClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingClassifier.html) with my multi-models, or try further with [XGBClassifier](https://xgboost.readthedocs.io/en/latest/python/python_api.html), or anything else that could do better than a simple mean.
- like challenge's TOP 3, integrate available data about age, male/female in the prediction model...
- give more chance to other transfer learning models, and __particularly try different parameters__: I spent most on my time with DenseNet and I kept the same parameters for others while it could be adapted for each of them...
- fine tune learning rate and other optimizer hyper parameters to speed up gradient descent (Adam with lr 0.002 often worked with earlier versions of the model)
- __detect the area of interest for each image through segmentation and zoom in on it__.
- give a try to other optimizers: AdaGrad, RMSProp, Adamax, Adadelta...
- try more with different batch_size to see how it impacts training (the final model reachs a GPU out of memory with a batch size of 64 [Google colab GPU])
- display images where the model fails to understand why it fails and maybe improve algorithm, data augmentation...  


---

I would also love to integrate some of the tools / algorithms I used througout this project into another framework like PyTorch, TensorFlow, Keras...
- moving fine-tuned cursor;
- cross validation;
- multi-crop strategy adapted to percentage of achieved objective;
- live plot while training;
- even cold start with smaller set of images or data augmentation that progressively adds planned transforms
- ...